In [1]:
pip install -r ../../requirements.txt


[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [3]:
%run -i ../../zkstats/core.py

In [4]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
comb_data_path = os.path.join('prover/comb_data.json')

=======================  ZK-STATS FLOW =======================

In [5]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')

f_raw_input = open(data_path, "r")
data = json.loads(f_raw_input.read())["input_data"][0]
data_tensor = torch.reshape(torch.tensor(data),(1, len(data), 1))

#  dummy data for data consumer: make the bound approx same as real data
dummy_data = np.random.uniform(min(data), max(data), len(data))
json.dump({"input_data":[dummy_data.tolist()]}, open(dummy_data_path, 'w'))

dummy_data_tensor = torch.reshape(torch.tensor(dummy_data), (1, len(dummy_data),1 ))
dummy_theory_output = torch.exp(torch.mean(torch.log(dummy_data_tensor)))



In [6]:
# Verifier/ data consumer side:
class verifier_model(nn.Module):
    def __init__(self):
        super(verifier_model, self).__init__()
        # w represents mean in this case
        self.w = nn.Parameter(data = dummy_theory_output, requires_grad = False)

    def forward(self,X):
        # some expression of tolerance to error in the inference
        return (torch.abs((torch.log(self.w)*X.size()[1])-torch.sum(torch.log(X)))<torch.log(torch.tensor(1.01)), self.w)

verifier_define_calculation(verifier_model, verifier_model_path, [dummy_data_path])

!@# model= <class '__main__.verifier_model'>
!@# circuit= verifier_model()
!@# circuit.eval= verifier_model()


/var/folders/t3/5psrvr1x0w1_6n9kx2n7d9700000gn/T/ipykernel_2464/2634772801.py:10: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  return (torch.abs((torch.log(self.w)*X.size()[1])-torch.sum(torch.log(X)))<torch.log(torch.tensor(1.01)), self.w)


In [7]:
# prover calculates settings, send to verifier
theory_output = torch.exp(torch.mean(torch.log(data_tensor)))
print("Theory output: ", theory_output)
class prover_model(nn.Module):
    def __init__(self):
        super(prover_model, self).__init__()
        # w represents mean in this case
        self.w = nn.Parameter(data = theory_output, requires_grad = False)

    def forward(self,X):
        # some expression of tolerance to error in the inference
        return (torch.abs((torch.log(self.w)*X.size()[1])-torch.sum(torch.log(X)))<torch.log(torch.tensor(1.01)), self.w)


prover_gen_settings([data_path], comb_data_path, prover_model,prover_model_path, [10], "resources", settings_path)

Theory output:  tensor(47.6981)
!@# model= <class '__main__.prover_model'>
!@# circuit= prover_model()
!@# circuit.eval= prover_model()
==== Generate & Calibrate Setting ====
scale:  [10]
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":10,"param_scale":10,"scale_rebase_multiplier":10,"lookup_range":[-16,180224],"logrows":18,"num_inner_cols":1,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private"},"num_rows":14432,"total_assignments":906,"total_const_size":0,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,10],"model_input_scales":[10],"module_sizes":{"kzg":[],"poseidon":[14432,[1]],"elgamal":[0,[0]]},"required_lookups":["Abs",{"Ln":{"scale":1024.0}},{"GreaterThan":{"a":0.0}}],"check_mode":"UNSAFE","version":"5.0.8","num_blinding_factors":null}


/var/folders/t3/5psrvr1x0w1_6n9kx2n7d9700000gn/T/ipykernel_2464/788883855.py:12: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  return (torch.abs((torch.log(self.w)*X.size()[1])-torch.sum(torch.log(X)))<torch.log(torch.tensor(1.01)), self.w)


In [8]:
# Here verifier & prover can concurrently call setup since all params are public to get pk.
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
verifier_setup(verifier_model_path, verifier_compiled_model_path, settings_path, srs_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
print("Theory output: ", theory_output)
prover_gen_proof(prover_model_path, comb_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path, srs_path)

spawning module 0


==== setting up ezkl ====


spawning module 2
spawning module 0
spawning module 2


Time setup: 41.862324953079224 seconds
Theory output:  tensor(47.6981)
!@# compiled_model exists? False
!@# compiled_model exists? True
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 47.6982421875
==== Generating Proof ====


spawning module 0
spawning module 2


proof:  {'instances': [[[14672592657000597156, 2402501382455789823, 5345685197495592745, 3011024057811024003], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [12156132031024860846, 9861177863383639274, 10629998272028772020, 2786626043251147566]]], 'proof': '1ccaa072eb70278669d8e47eae3cd1c3f2516cf6641ec4739b7c97ef4660d77e225ec15c8a2836d3cc709f01c59e225d497d1c0a14f43b41cfcf7393b7b474471c348aeaafa0d43e32c72d711c1adf89267d90c8dddfad961e9121a24f7593370e632e8c7e4ecd0a5503d3062954f079d850f80acf6dd9d7de180a783013288b087e5b6d9ababf3144d30d5b8841dff63652f1885974b40bbab2954c9151202105d17002032412a92f6bc64d5efe212b7103cdaafb9ac18ab5073b492f9981a02bc1ef30bf77a2f763a18a2d742f8c3d76c47fb6de10c1deea713000a8b15405073e3091808b25beff5b30789f25ec12323b5d4a0d9571f4eeb0caa0322809191c0263d1b732648240b6f1d50458dff5d33601138013f9bb76d2ec3fa0ee1df423dd681ff19b2ac7bc20f916ff0e5e20c2335a7c4774304d176a780d484edeb30101ffef70a45ee76e68ad33824d89f5b76496bd27713867a06baee2ff69da

In [9]:
# Verifier verifies
verifier_verify(proof_path, settings_path, vk_path, srs_path)

num_inputs:  1
prf instances:  [[[14672592657000597156, 2402501382455789823, 5345685197495592745, 3011024057811024003], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [12156132031024860846, 9861177863383639274, 10629998272028772020, 2786626043251147566]]]
proof boolean:  1.0
proof result 1 : 47.6982421875
verified
